In this walkthrough, we will develop a neural Sentiment Analysis model for Norwegian without access to labelled data. The first thing we will do is to read in our data and get it into Spacy DocBins:


In [1]:
import sys
import spacy
from spacy.tokens import DocBin
import pandas as pd
try:
    import transformers
except ImportError as e:
    print(e, ', which is required to run this notebook')

nlp = spacy.load("nb_core_news_md")

train_doc_bin = DocBin(store_user_data=True)
dev_doc_bin = DocBin(store_user_data=True)
test_doc_bin = DocBin(store_user_data=True)

train = pd.read_csv("../../data/sentiment/norec_sentence/train.txt", delimiter="\t", header=None)
dev = pd.read_csv("../../data/sentiment/norec_sentence/dev.txt", delimiter="\t", header=None)
test = pd.read_csv("../../data/sentiment/norec_sentence/test.txt", delimiter="\t", header=None)

We load the data as a `pandas.DataFrame`, where the first column is the sentence index, the second is the polarity (0=Negative, 1=Neutral, 2=Positive), and the third is the pretokenized sentence, as a string. We then create a Spacy DocBin object for train, dev, and test splits.

In [2]:
print(train.head(10))

   0                                                  1
0  1                                Stor og bred Seagal
1  1  Steven Seagal er blitt like stor som Travolta ...
2  1  I denne filmen handler det om en seriemorder m...
3  1   Dermed er genrens patologi-imperativ ivaretatt .
4  0                     Verre er det med slagsmålene .
5  0  Klipperen har overtatt Seagals martial art , o...
6  1                              Men noen fikse ting :
7  1  Hettemannen ser ut som David Beckham etter en ...
8  1                  Seagal får en Dirty Harry-pakke :
9  2  Ei finslig dame skal bli med på kjøret og obse...


In [3]:
for sid, (label, sent) in train.iterrows():
    doc = nlp(sent)
    doc.user_data["gold"] = label
    train_doc_bin.add(doc)
train_doc_bin.to_disk("../../data/sentiment/norec_sentence/train.docbin")

for sid, (label, sent) in dev.iterrows():
    doc = nlp(sent)
    doc.user_data["gold"] = label
    dev_doc_bin.add(doc)
dev_doc_bin.to_disk("../../data/sentiment/norec_sentence/dev.docbin")

for sid, (label, sent) in test.iterrows():
    doc = nlp(sent)
    doc.user_data["gold"] = label
    test_doc_bin.add(doc)
test_doc_bin.to_disk("../../data/sentiment/norec_sentence/test.docbin")


Next, we will begin to build up our labelling functions. We will start by training a document-level Bag-Of-Words (BOW) model on example Norwegian Agency for Exchange Cooperation (NoReC) documents. These are given star ratings from 1-6, so the labelling function will have to map these to negative, neutral, and positive.

In [4]:
import sys
sys.path.insert(0, '../..')
import skweak
from sklearn.metrics import f1_score
from sentiment_models import DocBOWAnnotator

dann = DocBOWAnnotator("doc_norec",
                       model_path="../../data/sentiment/models/doc",
                       doclevel_data="../../data/sentiment/conllu.tar.gz")

Fitting model on ../../data/sentiment/conllu.tar.gz
Doc-level F1: 0.313
Saving vectorizer and model to ../../data/sentiment/models/doc


Next, we will train a multi-lingual BERT model on the Stanford Sentiment Treebank.

In [5]:
from transformer_model import SSTDataLoader, train
from transformers import BertTokenizer, BertForSequenceClassification

train_loader = SSTDataLoader("../../data/sentiment/sst/train.txt")
dev_loader = SSTDataLoader("../../data/sentiment/sst/dev.txt")
test_loader = SSTDataLoader("../../data/sentiment/sst/test.txt")

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)
train(model, tokenizer, train_loader, dev_loader, "../../data/sentiment/models/mbert-sst")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

training for 20 epochs...


 16%|█▋        | 88/534 [05:41<28:53,  3.89s/it]


KeyboardInterrupt: 

We can construct a full annotator with all annotators described above, and then run it on the NoReC sentence-level dataset

In [6]:
from norec_sentiment import FullSentimentAnnotator

ann = FullSentimentAnnotator()
ann.add_all()

ann.annotate_docbin("../../data/sentiment/norec_sentence/train.docbin",
                    "../../data/sentiment/norec_sentence/train_pred.docbin")

ann.annotate_docbin("../../data/sentiment/norec_sentence/dev.docbin",
                    "../../data/sentiment/norec_sentence/dev_pred.docbin")

ann.annotate_docbin("../../data/sentiment/norec_sentence/test_pred.docbin",
                    "../../data/sentiment/norec_sentence/test_pred.docbin")

Loading lexicon functions
Loading learned sentiment model functions
Loaded model from ../../data/sentiment/models/doc
Loaded nlptown/bert-base-multilingual-uncased-sentiment
Loaded mBERT from ../../data/sentiment/models/mbert-sst
Number of processed documents: 1
Number of processed documents: 2
Number of processed documents: 3
Number of processed documents: 4
Number of processed documents: 5
Number of processed documents: 6
Number of processed documents: 7
Number of processed documents: 8
Number of processed documents: 9
Number of processed documents: 10
Number of processed documents: 11
Number of processed documents: 12
Number of processed documents: 13
Number of processed documents: 14
Number of processed documents: 15
Number of processed documents: 16
Number of processed documents: 17
Number of processed documents: 18
Number of processed documents: 19
Number of processed documents: 20
Number of processed documents: 21
Number of processed documents: 22
Number of processed documents: 

Number of processed documents: 232
Number of processed documents: 233
Number of processed documents: 234
Number of processed documents: 235
Number of processed documents: 236
Number of processed documents: 237
Number of processed documents: 238
Number of processed documents: 239
Number of processed documents: 240
Number of processed documents: 241
Number of processed documents: 242
Number of processed documents: 243
Number of processed documents: 244
Number of processed documents: 245
Number of processed documents: 246
Number of processed documents: 247
Number of processed documents: 248
Number of processed documents: 249
Number of processed documents: 250
Number of processed documents: 251
Number of processed documents: 252
Number of processed documents: 253
Number of processed documents: 254
Number of processed documents: 255
Number of processed documents: 256
Number of processed documents: 257
Number of processed documents: 258
Number of processed documents: 259
Number of processed 

Number of processed documents: 468
Number of processed documents: 469
Number of processed documents: 470
Number of processed documents: 471
Number of processed documents: 472
Number of processed documents: 473
Number of processed documents: 474
Number of processed documents: 475
Number of processed documents: 476
Number of processed documents: 477
Number of processed documents: 478
Number of processed documents: 479
Number of processed documents: 480
Number of processed documents: 481
Number of processed documents: 482
Number of processed documents: 483
Number of processed documents: 484
Number of processed documents: 485
Number of processed documents: 486
Number of processed documents: 487
Number of processed documents: 488
Number of processed documents: 489
Number of processed documents: 490
Number of processed documents: 491
Number of processed documents: 492
Number of processed documents: 493
Number of processed documents: 494
Number of processed documents: 495
Number of processed 

Number of processed documents: 703
Number of processed documents: 704
Number of processed documents: 705
Number of processed documents: 706
Number of processed documents: 707
Number of processed documents: 708
Number of processed documents: 709
Number of processed documents: 710
Number of processed documents: 711
Number of processed documents: 712
Number of processed documents: 713
Number of processed documents: 714
Number of processed documents: 715
Number of processed documents: 716
Number of processed documents: 717
Number of processed documents: 718
Number of processed documents: 719
Number of processed documents: 720
Number of processed documents: 721
Number of processed documents: 722
Number of processed documents: 723
Number of processed documents: 724
Number of processed documents: 725
Number of processed documents: 726
Number of processed documents: 727
Number of processed documents: 728
Number of processed documents: 729
Number of processed documents: 730
Number of processed 

Number of processed documents: 939
Number of processed documents: 940
Number of processed documents: 941
Number of processed documents: 942
Number of processed documents: 943
Number of processed documents: 944
Number of processed documents: 945
Number of processed documents: 946
Number of processed documents: 947
Number of processed documents: 948
Number of processed documents: 949
Number of processed documents: 950
Number of processed documents: 951
Number of processed documents: 952
Number of processed documents: 953
Number of processed documents: 954
Number of processed documents: 955
Number of processed documents: 956
Number of processed documents: 957
Number of processed documents: 958
Number of processed documents: 959
Number of processed documents: 960
Number of processed documents: 961
Number of processed documents: 962
Number of processed documents: 963
Number of processed documents: 964
Number of processed documents: 965
Number of processed documents: 966
Number of processed 

...

Number of processed documents: 955
Number of processed documents: 956
Number of processed documents: 957
Number of processed documents: 958
Number of processed documents: 959
Number of processed documents: 960
Number of processed documents: 961
Number of processed documents: 962
Number of processed documents: 963
Number of processed documents: 964
Number of processed documents: 965
Number of processed documents: 966
Number of processed documents: 967
Number of processed documents: 968
Number of processed documents: 969
Number of processed documents: 970
Number of processed documents: 971
Number of processed documents: 972
Number of processed documents: 973
Number of processed documents: 974
Number of processed documents: 975
Number of processed documents: 976
Number of processed documents: 977
Number of processed documents: 978
Number of processed documents: 979
Number of processed documents: 980
Number of processed documents: 981
Number of processed documents: 982
Number of processed 

Number of processed documents: 126
Number of processed documents: 127
Number of processed documents: 128
Number of processed documents: 129
Number of processed documents: 130
Number of processed documents: 131
Number of processed documents: 132
Number of processed documents: 133
Number of processed documents: 134
Number of processed documents: 135
Number of processed documents: 136
Number of processed documents: 137
Number of processed documents: 138
Number of processed documents: 139
Number of processed documents: 140
Number of processed documents: 141
Number of processed documents: 142
Number of processed documents: 143
Number of processed documents: 144
Number of processed documents: 145
Number of processed documents: 146
Number of processed documents: 147
Number of processed documents: 148
Number of processed documents: 149
Number of processed documents: 150
Number of processed documents: 151
Number of processed documents: 152
Number of processed documents: 153
Number of processed 

Number of processed documents: 362
Number of processed documents: 363
Number of processed documents: 364
Number of processed documents: 365
Number of processed documents: 366
Number of processed documents: 367
Number of processed documents: 368
Number of processed documents: 369
Number of processed documents: 370
Number of processed documents: 371
Number of processed documents: 372
Number of processed documents: 373
Number of processed documents: 374
Number of processed documents: 375
Number of processed documents: 376
Number of processed documents: 377
Number of processed documents: 378
Number of processed documents: 379
Number of processed documents: 380
Number of processed documents: 381
Number of processed documents: 382
Number of processed documents: 383
Number of processed documents: 384
Number of processed documents: 385
Number of processed documents: 386
Number of processed documents: 387
Number of processed documents: 388
Number of processed documents: 389
Number of processed 

Number of processed documents: 597
Number of processed documents: 598
Number of processed documents: 599
Number of processed documents: 600
Number of processed documents: 601
Number of processed documents: 602
Number of processed documents: 603
Number of processed documents: 604
Number of processed documents: 605
Number of processed documents: 606
Number of processed documents: 607
Number of processed documents: 608
Number of processed documents: 609
Number of processed documents: 610
Number of processed documents: 611
Number of processed documents: 612
Number of processed documents: 613
Number of processed documents: 614
Number of processed documents: 615
Number of processed documents: 616
Number of processed documents: 617
Number of processed documents: 618
Number of processed documents: 619
Number of processed documents: 620
Number of processed documents: 621
Number of processed documents: 622
Number of processed documents: 623
Number of processed documents: 624
Number of processed 

Number of processed documents: 832
Number of processed documents: 833
Number of processed documents: 834
Number of processed documents: 835
Number of processed documents: 836
Number of processed documents: 837
Number of processed documents: 838
Number of processed documents: 839
Number of processed documents: 840
Number of processed documents: 841
Number of processed documents: 842
Number of processed documents: 843
Number of processed documents: 844
Write to ../../data/sentiment/norec_sentence/test_pred.docbin...done


Now we can use the HMM model from skweak to aggregate all the predictions from the weak labelling functions.

In [7]:
unified_model = skweak.aggregation.HMM("hmm", [0, 1, 2], sequence_labelling=False)
unified_model.fit("../../data/sentiment/norec_sentence/train_pred.docbin")
unified_model.annotate_docbin("../../data/sentiment/norec_sentence/train_pred.docbin",
                              "../../data/sentiment/norec_sentence/train_pred.docbin")

unified_model.annotate_docbin("../../data/sentiment/norec_sentence/dev_pred.docbin",
                              "../../data/sentiment/norec_sentence/dev_pred.docbin")

unified_model.annotate_docbin("../../data/sentiment/norec_sentence/test_pred.docbin",
                              "../../data/sentiment/norec_sentence/test_pred.docbin")

KeyError: 'IS_ALPHA'

We can also compare with a majority voting aggregator, which will highlight the benefits of skweak.

In [ ]:
mv = skweak.aggregation.MajorityVoter("mv", [0, 1, 2], sequence_labelling=False) #type: ignore
mv.annotate_docbin("../../data/sentiment/norec_sentence/test_pred.docbin",
                   "../../data/sentiment/norec_sentence/test_pred.docbin")

Now, we can evaluate everything

In [ ]:
pred_docs = list(utils.docbin_reader("../../data/sentiment/norec_sentence/test_pred.docbin"))
gold = [d.user_data["gold"] for d in pred_docs]

# Evaluate the weak labelling approaches themselves
for lexicon in pred_docs[0].user_data["spans"].keys():
    pred = []
    for d in pred_docs:
        for span in d.spans[lexicon]:
            pred.append(span.label_)

    lex_f1 = f1_score(gold, pred, average="macro")
    print("{0}:\t{1:.3f}".format(lexicon, lex_f1))
    
for aggregator in ["mv", "hmm"]:
    pred = []
    for d in pred_docs:
        for span in d.spans[aggregator]:
            pred.append(span.label_)
    agg_f1 = f1_score(gold, pred, average="macro")
    print("{0}:\t{1:.3f}".format(aggregator, agg_f1))
